In [1]:
import sys
import os
import jax
jax.config.update("jax_enable_x64", True)

# Add the models directory to the Python path
sys.path.append(os.path.join(os.getcwd(),'..', 'models'))

# Import the functions from transport_analytical.py
from transport_analytical import constant_injection, pulse_injection
from transport_analytical_jax import constant_injection as const_injection_jax
from transport_analytical_jax import pulse_injection as pulse_injection_jax
from transport_analytical_sympy import vec_pulse, vec_par_pulse, sympy_loss, sympy_jac
import juliacall

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [2]:
%%julia
include("testing_julia_code.jl")
using Optim
using OptimizationOptimJL
using Optimization
using SciMLSensitivity
using CSV
using DataFrames

## Checking the correctness and efficiency of optimization of the parameters of a solution

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import timeit
import jax
import jax.scipy as jscipy
from jax.scipy.optimize import minimize as jminimize

In [4]:
data = np.loadtxt('data/data_a.csv', delimiter=',', skiprows=1)
t = data[:,0]
c = data[:,1]

### Pure Numpy function

In [8]:
c0 = 2
c_in = 0
v = 1e-5
alpha_l = 1e-3
De = 2.01e-9
Dl = De + alpha_l*v
t_pulse = 3600
col_length = 0.121
x = np.array([col_length])
c_model = np.zeros((len(t), len(x)))
Q0_ml = 6 #ml/hr
Q0 = Q0_ml*1e-6/3600 # m3/s
diam = 0.037 # diameter of the column [m]
area = np.pi*(diam/2)**2 # cross-sectional area of the column [m2]
q = Q0/area # Darcy velocity [m/s]
def loss_numpy(p):
    phi = p[0]
    alpha_l = p[1]
    v = q/phi
    Dl = 2.01e-9 + alpha_l*v #bromide diffusion + dispersion
    pulse_injection(c_model, x, t, c0, c_in, v, Dl, t_pulse)
    return np.sum((c_model.squeeze() - c)**2)


In [9]:
%timeit loss_numpy([0.3, 1e-3])

28.1 μs ± 274 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
def loss(p):
    return sympy_loss(c, x, t, c0, c_in, p[0], q, De, p[1], t_pulse)

def jac(p):
    return sympy_jac(c, x, t, c0, c_in, p[0], q, De, p[1], t_pulse)

%timeit loss([0.3, 1e-3])

84 μs ± 42.3 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
print(loss_numpy([0.3, 1e-3]))
print(loss([0.3, 1e-3]))

0.2237010193202472
0.22370101932024727


In [12]:
@jax.jit
def loss_jax(p):
    phi = p[0]
    alpha_l = p[1]
    v = q/phi
    Dlj = 2.01e-9 + alpha_l*v #bromide diffusion + dispersion
    c_model = pulse_injection_jax(x, t, c0, c_in, v, Dlj, t_pulse)
    return np.sum((c_model.squeeze() - c)**2)

In [13]:
np.sum((c_model.squeeze() - c)**2)

0.2237010193202472

In [15]:
%timeit loss_jax([0.3, 1e-3])

5.52 μs ± 42.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [16]:
loss_jax(np.array([0.3, 1e-3]))

Array(0.22370102, dtype=float64)

In [17]:
%timeit resnp = scipy.optimize.minimize(loss_numpy, [0.3, 1e-3], method='L-BFGS-B', tol=1e-8)

3.39 ms ± 28.6 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
resnp = scipy.optimize.minimize(loss_numpy, [0.3, 1e-3], method='L-BFGS-B', tol=1e-8)
resnp

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 0.07323841473894227
        x: [ 2.834e-01  1.149e-03]
      nit: 14
      jac: [ 3.341e-05 -8.526e-04]
     nfev: 63
     njev: 21
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>

In [20]:
%timeit ressp = scipy.optimize.minimize(loss, [0.3, 1e-3],jac=jac, method='L-BFGS-B', tol=1e-8)

4.02 ms ± 335 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
ressp = scipy.optimize.minimize(loss, [0.3, 1e-3],jac=jac, method='L-BFGS-B', tol=1e-8)
ressp

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 0.07323841473018737
        x: [ 2.834e-01  1.149e-03]
      nit: 15
      jac: [ 2.116e-05  1.725e-04]
     nfev: 22
     njev: 22
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>

In [26]:
%timeit resjax = jminimize(loss_jax, np.array([0.3, 1e-3]), method="BFGS", tol=1e-8)

909 ms ± 14.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
resjax = jminimize(loss_jax, np.array([0.3, 1e-3]), method="BFGS", tol=1e-8)
resjax

OptimizeResults(x=Array([0.28163161, 0.00149976], dtype=float64), success=Array(False, dtype=bool), status=Array(3, dtype=int64, weak_type=True), fun=Array(0.0906218, dtype=float64), jac=Array([-3.2168285, 71.8676247], dtype=float64), hess_inv=Array([[1.73547405e-03, 1.05111040e-04],
       [1.05111040e-04, 1.23767756e-05]], dtype=float64), nfev=Array(12, dtype=int64, weak_type=True), njev=Array(12, dtype=int64, weak_type=True), nit=Array(3, dtype=int64, weak_type=True))

In [29]:
%timeit resjaxsci = scipy.optimize.minimize(loss_jax, np.array([0.3, 1e-3]),jac=jax.grad(loss_jax), method='L-BFGS-B', tol=1e-8)

32.4 ms ± 573 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [30]:
resjaxsci = scipy.optimize.minimize(loss_jax, np.array([0.3, 1e-3]),jac=jax.grad(loss_jax), method='L-BFGS-B', tol=1e-8)
resjaxsci

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 0.07323841473110974
        x: [ 2.834e-01  1.149e-03]
      nit: 14
      jac: [ 3.154e-05 -8.005e-04]
     nfev: 21
     njev: 21
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>

In [31]:
%%julia
data = CSV.read("data/data_a.csv",DataFrame)
t = data[:,1]
c = data[:,2]
c0 = 2.0
c_in = 0.0

t_pulse = 3600.0
x = [0.121]
Q0_ml = 6 #ml/hr
Q0 = Q0_ml*1e-6/3600 # m3/s
diam = 0.037 # diameter of the column [m]
area = pi*(diam/2)^2 # cross-sectional area of the column [m2]
q = Q0/area # specific discharge [m/s]
De = 2.01e-9
c_model = zeros(length(t), length(x))
v_ = q/0.3
Dl_ = De + 1e-3*v_

7.176948887002528e-09

In [36]:
%%julia
adtype = Optimization.AutoForwardDiff()
optf = Optimization.OptimizationFunction((u,p)->loss_julia(u, x, t, c0, c_in, q, De, t_pulse), adtype)
lb = [0.1, 1e-4]
ub = [0.7, 0.1]
p0 = [0.3, 1e-3]
optprob = Optimization.OptimizationProblem(optf, p0,)
@btime result_ode = Optimization.solve(optprob, Optim.LBFGS())


  385.500 μs (2627 allocations: 235.06 KiB)


retcode: Success
u: 2-element Vector{Float64}:
 0.2834378530055382
 0.0011485912307908656

In [35]:
%%julia
best_p = result_ode.u
loss_julia(best_p, x, t, c0, c_in, q, De, t_pulse)

0.07323841472980072